In [ ]:
import polars as pl
import numpy as np

# Constants
num_items = 5000
num_suppliers = 20
num_countries = 5
num_factory_demand_rows = 45000
num_target_volume_rows = num_suppliers * num_countries

# Create dummy ITEM LIST dataset
item_list_data = {
    'PDM MTL NUM': [f'MTL{i:04d}' for i in range(1, num_items + 1)],
    'NAME': [f'Item{i:04d}' for i in range(1, num_items + 1)],
    'ITEM TYPE': [f'Type{i%10}' for i in range(1, num_items + 1)],
    'STATE': ['New'] * num_items,
    'PDM SPL MTL NAME': [f'Material{i:04d}' for i in range(1, num_items + 1)],
    'PCX SPL MTL ID': [f'PCX{i:04d}' for i in range(1, num_items + 1)],
    'PCX MTL ID': [f'PCXM{i:04d}' for i in range(1, num_items + 1)],
    'SUPPLIER': [f'Supplier{i%num_suppliers + 1}' for i in range(1, num_items + 1)],
    'SUPPLIER COUNTRY': [f'Country{i%num_countries + 1}' for i in range(1, num_items + 1)],
    'PDM SPL MTL ': [f'SPLMTL{i:04d}' for i in range(1, num_items + 1)],
    'PP TYPE': [f'Type{i%3 + 1}' for i in range(1, num_items + 1)],
    'CAPACITY CODE': [f'Cap{i%3 + 1}' for i in range(1, num_items + 1)]
}
item_list_df = pl.DataFrame(item_list_data)

# Create dummy PRICE LIST dataset
price_list_data = {
    'PDM MTL NUM': [f'MTL{i:04d}' for i in range(1, num_items + 1)],
    'ITEM': [f'Item{i:04d}' for i in range(1, num_items + 1)],
    'SUPPLIER': [f'Supplier{i%num_suppliers + 1}' for i in range(1, num_items + 1)],
    'PDM SPL MTL NAME': [f'Material{i:04d}' for i in range(1, num_items + 1)],
    'PCX SPL MTL ID': [f'PCX{i:04d}' for i in range(1, num_items + 1)],
    'PCX MTL ID': [f'PCXM{i:04d}' for i in range(1, num_items + 1)],
    'BID PRICE': np.random.uniform(10, 100, num_items),
    'SUPPLIER_NAME': [f'Supplier{i%num_suppliers + 1}' for i in range(1, num_items + 1)],
    'SUPPLIER COUNTRY': [f'Country{i%num_countries + 1}' for i in range(1, num_items + 1)],
    'PDM SPL MTL NUM': [f'SPLMTL{i:04d}' for i in range(1, num_items + 1)],
    'PP TYPE': [f'Type{i%3 + 1}' for i in range(1, num_items + 1)],
    'CAPACITY CODE': [f'Cap{i%3 + 1}' for i in range(1, num_items + 1)]
}
price_list_df = pl.DataFrame(price_list_data)

# Create dummy FACTORY DEMAND dataset
factory_demand_data = {
    'PDM MTL NUM': [f'MTL{i%num_items + 1:04d}' for i in range(1, num_factory_demand_rows + 1)],
    'PCX MTL ID': [f'PCXM{i%num_items + 1:04d}' for i in range(1, num_factory_demand_rows + 1)],
    'VOLUME': np.random.choice([0, 1000, 2000], num_factory_demand_rows),
    'FACTORY CODE': [f'Factory{i%60 + 1}' for i in range(1, num_factory_demand_rows + 1)]
}
factory_demand_df = pl.DataFrame(factory_demand_data)

# Create dummy TARGET VOLUME dataset
target_volume_data = {
    'SUPPLIER': [f'Supplier{i%num_suppliers + 1}' for i in range(1, num_target_volume_rows + 1)],
    'SUPPLIER COUNTRY': [f'Country{i%num_countries + 1}' for i in range(1, num_target_volume_rows + 1)],
    'TARGET %': np.random.uniform(0.01, 0.05, num_target_volume_rows),
    'TARGET % RANGE': np.random.uniform(0.01, 0.02, num_target_volume_rows),
    'SUPPLIER TYPE': [f'Type{i%3 + 1}' for i in range(1, num_target_volume_rows + 1)],
    'TOTAL VOLUME': np.random.randint(10000, 50000, num_target_volume_rows)
}
target_volume_df = pl.DataFrame(target_volume_data)

# Calculate MIN TARGET VOLUME and MAX TARGET VOLUME
target_volume_df = target_volume_df.with_columns([
    ((target_volume_df['TARGET %'] - target_volume_df['TARGET % RANGE']) * target_volume_df['TOTAL VOLUME']).alias('MIN TARGET VOLUME'),
    ((target_volume_df['TARGET %'] + target_volume_df['TARGET % RANGE']) * target_volume_df['TOTAL VOLUME']).alias('MAX TARGET VOLUME')
])

# Merge ITEM LIST with PRICE LIST on 'PDM MTL NUM'
merged_df = item_list_df.join(price_list_df, on='PDM MTL NUM', how='inner')

# Prioritize by lower price
merged_df = merged_df.sort(by='BID PRICE')

# Apply LOCAL FOR LOCAL logic (matching supplier and factory country)
merged_df = merged_df.with_column((merged_df['SUPPLIER_x'] == merged_df['SUPPLIER COUNTRY_y']).alias('LOCAL_FOR_LOCAL'))
merged_df = merged_df.sort(by=['BID PRICE', 'LOCAL_FOR_LOCAL'], reverse=[False, True])

# Define a function for allocation based on target volume and factory demand
def allocate_volume(merged_df, target_volume_df, factory_demand_df):
    allocations = []
    
    for supplier in target_volume_df.iter_rows(named=True):
        supplier_name = supplier['SUPPLIER']
        min_volume = supplier['MIN TARGET VOLUME']
        max_volume = supplier['MAX TARGET VOLUME']
        supplier_df = merged_df.filter(pl.col('SUPPLIER_x') == supplier_name)
        
        allocated_volume = 0
        for row in supplier_df.iter_rows(named=True):
            if allocated_volume >= max_volume:
                break

            factory_options = factory_demand_df.filter(pl.col('PDM MTL NUM') == row['PDM MTL NUM'])
            for factory in factory_options.iter_rows(named=True):
                if allocated_volume >= max_volume:
                    break
                if factory['VOLUME'] > 0:
                    volume_to_allocate = min(max_volume - allocated_volume, factory['VOLUME'])
                    allocations.append({
                        'PDM MTL NUM': row['PDM MTL NUM'],
                        'SUPPLIER': row['SUPPLIER_x'],
                        'FACTORY CODE': factory['FACTORY CODE'],
                        'ALLOCATION (UNITS)': volume_to_allocate,
                        'PRICE': row['BID PRICE']
                    })
                    allocated_volume += volume_to_allocate
                    factory['VOLUME'] -= volume_to_allocate
    
    return pl.DataFrame(allocations)

# Perform the allocation
allocation_df = allocate_volume(merged_df, target_volume_df, factory_demand_df)

# Output the allocation results in the same format as ALLOCATION RESULT file
allocation_result_columns = ['PDM MTL NUM', 'SUPPLIER', 'FACTORY CODE', 'ALLOCATION (UNITS)', 'PRICE']
allocation_df = allocation_df.select(allocation_result_columns)

# Save the results to an Excel file
allocation_df.write_excel('ALLOCATION_RESULT.xlsx')

print("Allocation results saved to ALLOCATION_RESULT.xlsx")